In [1]:
import pandas as pd
import csv
import collections
import re
import pickle
import numpy as np

In [2]:
fileNeop = "../data/RTRT_NEOPLASI_corr.csv"
fileIsto = "../data/ISTOLOGIE_corr.csv" 
fileIcdo = "icdo-3-dict-en.p"

fileSaveCorpus = "corpusTemporal/corpusTemporal.p"
fileSaveValues = "corpusTemporal/valuesTemporal.p"
fileSaveTrainWordVectors = "corpusTemporal/trainWordVectors.txt"

In [3]:
encoding = 'iso-8859-1'

In [4]:
icdo = pickle.load(open(fileIcdo, 'rb'))

# Load neop

In [5]:
dfNeop = pd.read_csv(fileNeop, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False,
                     usecols = ['id_neopl', 'sede_icdo3',
                                'morfologia_icdo3', 'inserimento_data_neoplasi'],
                     dtype={'id_neopl':pd.np.float64, 'sede_icdo3':pd.np.str, 
                            'morfologia_icdo3':pd.np.str, 'inserimento_data_neoplasi':pd.np.str},
                    #parse_dates=['inserimento_data_neoplasi'], 
                    #date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
                    )

In [6]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [11]:
anni = collections.Counter(dfNeop.inserimento_data_neoplasi.apply(lambda d: str(d)[:4]))
for k in sorted(anni):
    print("{} -> {}".format(k, anni[k]))

1990 -> 5584
1991 -> 5455
1992 -> 2958
1993 -> 5812
1994 -> 6046
1995 -> 1497
1996 -> 6623
1997 -> 12163
1998 -> 11394
1999 -> 16572
2000 -> 7471
2001 -> 12505
2002 -> 7207
2003 -> 9074
2004 -> 11776
2005 -> 12918
2006 -> 17107
2007 -> 30144
2008 -> 14649
2009 -> 3550
2010 -> 8264
2011 -> 6271
2012 -> 43362
2013 -> 1010
2014 -> 13
nan -> 50427


In [7]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [8]:
dfNeop = dfNeop.dropna('index')

In [9]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [10]:
dfNeop.dtypes

id_neopl                     float64
inserimento_data_neoplasi     object
sede_icdo3                    object
morfologia_icdo3              object
dtype: object

In [11]:
dfNeop['inserimento_data_neoplasi'] =  pd.to_datetime(dfNeop['inserimento_data_neoplasi'], format='%Y-%m-%d %H:%M:%S')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
0,1.0,2002-09-06 00:00:00,C060,80513
1,2.0,2002-12-16 00:00:00,C504,84803
2,3.0,2002-12-11 00:00:00,C509,85202
3,4.0,2002-12-11 00:00:00,C443,80903
4,5.0,2003-04-28 00:00:00,C259,81403
...,...,...,...,...
309847,346854.0,2014-02-13 08:51:32,C509,85003
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001


In [13]:
dfNeop.dtypes

id_neopl                            float64
inserimento_data_neoplasi    datetime64[ns]
sede_icdo3                           object
morfologia_icdo3                     object
dtype: object

In [14]:
dfNeop = dfNeop.sort_values(by=['inserimento_data_neoplasi'])

In [15]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3
107276,108110.0,1990-04-20 00:00:00,C443,80003
78576,79172.0,1990-04-20 00:00:00,C341,80003
78595,79191.0,1990-04-24 00:00:00,C329,80703
78602,79198.0,1990-04-24 00:00:00,C509,85203
78601,79197.0,1990-04-24 00:00:00,C710,94003
...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002
309849,346856.0,2014-02-14 11:53:40,C424,97651
309850,346857.0,2014-02-14 12:00:33,C619,80001
295873,346866.0,2014-03-10 12:26:26,C169,81403


remove invalid sites

In [16]:
dfNeop['correctSede'] = dfNeop.sede_icdo3.map(lambda s: len(s)==4 and s[0] == "C" and int(s[1:3]) in icdo.keys())

In [17]:
dfNeop = dfNeop[dfNeop.correctSede]

In [18]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede
107276,108110.0,1990-04-20 00:00:00,C443,80003,True
78576,79172.0,1990-04-20 00:00:00,C341,80003,True
78595,79191.0,1990-04-24 00:00:00,C329,80703,True
78602,79198.0,1990-04-24 00:00:00,C509,85203,True
78601,79197.0,1990-04-24 00:00:00,C710,94003,True
...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True
309849,346856.0,2014-02-14 11:53:40,C424,97651,True
309850,346857.0,2014-02-14 12:00:33,C619,80001,True
295873,346866.0,2014-03-10 12:26:26,C169,81403,True


In [19]:
dfNeop['y'] = dfNeop.sede_icdo3.map(lambda s: int(s[1:3]))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
dfNeop

,id_neopl,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y
107276,108110.0,1990-04-20 00:00:00,C443,80003,True,44
78576,79172.0,1990-04-20 00:00:00,C341,80003,True,34
78595,79191.0,1990-04-24 00:00:00,C329,80703,True,32
78602,79198.0,1990-04-24 00:00:00,C509,85203,True,50
78601,79197.0,1990-04-24 00:00:00,C710,94003,True,71
...,...,...,...,...,...,...
309848,346855.0,2014-02-13 11:20:50,C509,85002,True,50
309849,346856.0,2014-02-14 11:53:40,C424,97651,True,42
309850,346857.0,2014-02-14 12:00:33,C619,80001,True,61
295873,346866.0,2014-03-10 12:26:26,C169,81403,True,16


# Load isto

In [21]:
dfIsto = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [22]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [23]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfIsto['notizie'] = dfIsto.notizie.map(nullToEmpty)
dfIsto['diagnosi'] = dfIsto.diagnosi.map(nullToEmpty)
dfIsto['macroscopia'] = dfIsto.macroscopia.map(nullToEmpty)

In [24]:
dfIsto = dfIsto.dropna('index')

In [25]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril..."
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.
...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...


In [26]:
dfIsto['text'] = dfIsto[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
dfIsto['text'] = dfIsto['text'].astype('unicode')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
dfIsto['text'] = dfIsto['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...
...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...


In [30]:
dfIsto['correctText'] = dfIsto.text.map(lambda t: len(t) > 0)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
dfIsto = dfIsto[dfIsto.correctText]

In [32]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


In [33]:
dfIsto = dfIsto.drop_duplicates(['text'])

In [34]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
173,174.0,328237.0,NEOPLASIA 2 CM Q.S.I. DX: MICROCALCIFICAZIONI ...,CARCINOMA DUTTALE IN SITU DELLA MAMMELLA. B5/L...,1) FRUSTOLI MAMMARI. 2) FRUSTOLI MAMMARI CON M...,NEOPLASIA 2 CM Q . S . I . DX : MICROCALCIFICA...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


In [35]:
dfIsto = dfIsto.drop_duplicates(['id_neopl'], keep=False)

In [36]:
dfIsto

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
107,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True
380,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True
418,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
550,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True
684,685.0,341032.0,,CARCINOMA BASOCELLULARE; MARGINI DI SEZIONE LI...,"1-2) Losanga di cute di cm 2.5, lesione irrego...",CARCINOMA BASOCELLULARE ; MARGINI DI SEZIONE L...,True
...,...,...,...,...,...,...,...
1592357,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True
1592361,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True
1592364,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True
1592369,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True


# Merge

In [37]:
dfMerge = pd.merge(dfIsto, dfNeop, on='id_neopl', validate="one_to_one")

In [38]:
dfMerge

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText,inserimento_data_neoplasi,sede_icdo3,morfologia_icdo3,correctSede,y
0,108.0,239358.0,,CARCINOMA A CELLULE BASALI. LESIONE COMPRESA ...,"Losanga cutanea di cm. 2,5x1,5 con lesione ril...",CARCINOMA A CELLULE BASALI . LESIONE COMPRESA ...,True,2008-03-06 14:14:39,C449,80903,True,44
1,381.0,344832.0,,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,Formazione nodulare della regione pettorale de...,CARCINOMA BASOCELLULARE DI TIPO ADENOIDE AVVOL...,True,2012-11-14 10:01:56,C449,80903,True,44
2,419.0,205046.0,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True,2007-02-09 10:07:13,C779,96513,True,77
3,551.0,340975.0,LESIONE VEGETANTE SESSILE DEL RETTO.,ADENOMA TUBULO-VILLOSO INTESTINALE CON ESTESE ...,BX SU LESIONE VEGETANTE DEL RETTO.,LESIONE VEGETANTE SESSILE DEL RETTO . ADENOMA ...,True,2012-10-10 10:19:28,C209,81401,True,20
4,685.0,341032.0,,CARCINOMA BASOCELLULARE; MARGINI DI SEZIONE LI...,"1-2) Losanga di cute di cm 2.5, lesione irrego...",CARCINOMA BASOCELLULARE ; MARGINI DI SEZIONE L...,True,2012-10-10 14:21:28,C449,80903,True,44
...,...,...,...,...,...,...,...,...,...,...,...,...
85170,1792424.0,346815.0,,CARCINOMA BASOCELLULARE; MARGINI NEGATIVI.,Frammento cutaneo della regione temporale dest...,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI . F...,True,2013-10-18 16:23:09,C443,80903,True,44
85171,1792428.0,337273.0,,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,1-2) Nr. 2 polipi peduncolati del colon discen...,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,True,2012-09-10 16:00:08,C186,80003,True,18
85172,1792431.0,341607.0,,CARCINOMA A CELLULE SQUAMOSE LARINGEO (A-B).,A) Vari frammenti friabili del terzo anteriore...,CARCINOMA A CELLULE SQUAMOSE LARINGEO ( A - B ...,True,2012-10-15 11:56:30,C329,80703,True,32
85173,1792436.0,344486.0,,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,Frammento cutaneo della gamba dx con lesione d...,MELANOMA CUTANEO TIPO DIFFUSIONE SUPERFICIALE ...,True,2012-11-12 10:35:31,C447,87432,True,44


# Unlabelled

In [39]:
dfUn = pd.read_csv(fileIsto, header = 0, encoding=encoding, quoting=csv.QUOTE_NONNUMERIC, low_memory=False, 
                     usecols = ['notizie', 'diagnosi', 'macroscopia', 'id_neopl', 'id_fonte'],
                     dtype={'id_neopl':pd.np.float64, 'id_fonte':pd.np.float64})

In [40]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [41]:
dfUn = dfUn[~dfUn.id_fonte.isin(dfMerge.id_fonte)]

In [42]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia
0,1.0,NaN,NaN,NaN,NaN
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.
2,3.0,NaN,NaN,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM."
3,4.0,NaN,NaN,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...
4,5.0,NaN,NaN,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,NaN
...,...,...,...,...,...
1592380,1792447.0,NaN,NaN,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...
1592381,1792448.0,NaN,NaN,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...
1592383,1792450.0,NaN,NaN,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...


In [43]:
nullToEmpty = lambda s: "" if pd.isnull(s) else str(s)
dfUn['notizie'] = dfUn.notizie.map(nullToEmpty)
dfUn['diagnosi'] = dfUn.diagnosi.map(nullToEmpty)
dfUn['macroscopia'] = dfUn.macroscopia.map(nullToEmpty)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [44]:
dfUn['text'] = dfUn[['notizie' ,'diagnosi', 'macroscopia']].apply(lambda t: " ".join(t).upper(), axis=1)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
dfUn['text'] = dfUn['text'].astype('unicode')

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
dfUn['text'] = dfUn['text'].apply(lambda t: " ".join(re.findall(r"\w+|[^\w\s]", t, re.UNICODE)))

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
dfUn['correctText'] = dfUn.text.map(lambda t: len(t) > 0)

/home/trianam/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
dfUn = dfUn[dfUn.correctText]

In [49]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...,True
2,3.0,NaN,,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM.",ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...,True
3,4.0,NaN,,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...,True
4,5.0,NaN,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
5,6.0,NaN,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO (vedi es...,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...,True
...,...,...,...,...,...,...,...
1592380,1792447.0,NaN,,"RESIDUE, MINIME ESPRESSIONI DI MALIGNITA' EPIT...",Biopsia estemporanea al microtomo congelatore:...,"RESIDUE , MINIME ESPRESSIONI DI MALIGNITA ' EP...",True
1592381,1792448.0,NaN,,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,True
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...,True
1592383,1792450.0,NaN,,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...,True


In [50]:
dfUn = dfUn[~dfUn.text.isin(dfMerge.text)]

In [51]:
dfUn

,id_fonte,id_neopl,notizie,diagnosi,macroscopia,text,correctText
1,2.0,NaN,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA.,PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE D...,ESAME ISTOLOGICO AL CONGELATORE.,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...,True
2,3.0,NaN,,ADENOMA TUBULARE INTESTINALE.,"POLIPO DEL SIGMA, DIAMETRO MAGGIORE 8 MM.",ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...,True
3,4.0,NaN,,FIBROLIPOMA (1-2),(1-2)Frammenti di aspetto lipomatoso di 3cm co...,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...,True
4,5.0,NaN,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...,True
5,6.0,NaN,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO (vedi es...,,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...,True
...,...,...,...,...,...,...,...
1592379,1792446.0,NaN,,ADENOMA TUBULARE INTESTINALE DI ALTO GRADO; BA...,"Polipo del colon ascendente (cm 0,7). (eseguit...",ADENOMA TUBULARE INTESTINALE DI ALTO GRADO ; B...,True
1592381,1792448.0,NaN,,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,Biopsia delle cavita' nasali. (eseguita colora...,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...,True
1592382,1792449.0,NaN,E.R.: tessuto prostatico residuo con caratteri...,ADENOCARCINOMA PROSTATICO A GRADO DI DIFFERENZ...,Prelievi lobo dx: 1) zona periferica paramedia...,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...,True
1592383,1792450.0,NaN,,CARCINOMA A CELLULE SQUAMOSE (G2) CON ESPRESSI...,Frammento cutaneo del padiglione auricolare si...,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...,True


# Final

In [52]:
dfFinal = dfMerge[['inserimento_data_neoplasi','text', 'y']]

In [53]:
dfFinal = dfFinal.rename(columns={"inserimento_data_neoplasi": "data"})

In [54]:
dfFinal = dfFinal.sort_values(by=['data'])

In [55]:
dfFinal

,data,text,y
61615,1992-04-13 00:00:00,METASTASI EPATICHE DA SOSPETTA NEOPLASIA COLON...,16
70999,1992-04-15 00:00:00,PSA : 1160 ; AREA IPOECOGENA LOBO SINISTRO ; I...,75
46748,1996-03-13 00:00:00,CARCINOMA DUTTALE INFILTRANTE ( NOS ) ( G2 ) D...,50
17030,1997-03-04 00:00:00,LESIONE PIGMENTATA DISCROMICA E PALPABILE OD :...,44
67580,1997-09-15 00:00:00,AREE DI ADENOCARCINOMA PROSTATICO A MEDIO GRAD...,23
...,...,...,...
84732,2014-02-13 08:51:32,CARCINOMA DUTTALE INFILTRANTE ( G2 ) DELLA MAM...,50
47068,2014-02-13 11:20:50,"CARCINOMA DUTTALE IN SITU NAS , PRESENZA DI CA...",50
78074,2014-02-14 12:00:33,1 - AGOBIOPSIA PROSTATICA DELLA LUNGHEZZA COMP...,61
1000,2014-03-10 12:26:26,CA . GASTRICO . ADENOCARCINOMA GASTRICO MODERA...,16


In [56]:
dfUnlabelled = dfUn[['text']]

In [57]:
dfUnlabelled

,text
1,PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLI...
2,ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGM...
3,FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI A...
4,QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO CO...
5,QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI E...
...,...
1592379,ADENOMA TUBULARE INTESTINALE DI ALTO GRADO ; B...
1592381,CIRCOSCRITTE ESPRESSIONI RIFERIBILI A CARCINOM...
1592382,E . R . : TESSUTO PROSTATICO RESIDUO CON CARAT...
1592383,CARCINOMA A CELLULE SQUAMOSE ( G2 ) CON ESPRES...


In [58]:
c = collections.Counter(dfFinal.y)
c.most_common()

[(44, 20994),
 (50, 9510),
 (18, 9247),
 (61, 6738),
 (67, 5514),
 (34, 4511),
 (16, 3788),
 (42, 3107),
 (20, 2838),
 (64, 1953),
 (54, 1895),
 (53, 1729),
 (56, 1187),
 (32, 935),
 (73, 928),
 (25, 846),
 (19, 782),
 (77, 703),
 (71, 676),
 (22, 624),
 (49, 554),
 (80, 482),
 (15, 323),
 (24, 317),
 (26, 304),
 (62, 297),
 (70, 296),
 (17, 246),
 (2, 241),
 (51, 238),
 (23, 228),
 (21, 224),
 (6, 217),
 (38, 200),
 (75, 171),
 (66, 157),
 (48, 142),
 (65, 138),
 (7, 134),
 (9, 132),
 (0, 130),
 (30, 104),
 (72, 104),
 (68, 104),
 (11, 98),
 (60, 84),
 (40, 76),
 (41, 76),
 (69, 74),
 (57, 66),
 (5, 61),
 (10, 59),
 (8, 59),
 (31, 57),
 (55, 53),
 (74, 47),
 (1, 47),
 (52, 44),
 (12, 43),
 (4, 43),
 (13, 36),
 (3, 34),
 (37, 30),
 (14, 24),
 (76, 24),
 (47, 22),
 (63, 19),
 (33, 6),
 (58, 4),
 (39, 1)]

In [59]:
testLen = int(len(dfFinal)/100*20)

In [60]:
dfTest = dfFinal.tail(testLen)

In [61]:
dfTrain = dfFinal.head(len(dfFinal) - testLen)

In [62]:
dfValid = dfTrain.tail(testLen)

In [63]:
dfTrain = dfTrain.head(len(dfTrain) - testLen)

In [64]:
dfTrain

,data,text,y
61615,1992-04-13 00:00:00,METASTASI EPATICHE DA SOSPETTA NEOPLASIA COLON...,16
70999,1992-04-15 00:00:00,PSA : 1160 ; AREA IPOECOGENA LOBO SINISTRO ; I...,75
46748,1996-03-13 00:00:00,CARCINOMA DUTTALE INFILTRANTE ( NOS ) ( G2 ) D...,50
17030,1997-03-04 00:00:00,LESIONE PIGMENTATA DISCROMICA E PALPABILE OD :...,44
67580,1997-09-15 00:00:00,AREE DI ADENOCARCINOMA PROSTATICO A MEDIO GRAD...,23
...,...,...,...
51629,2010-12-30 11:37:10,COMPONENTE MONOCLONALE IGA . OD : MIELOMA . MI...,42
49858,2010-12-30 11:37:10,CARCINOMA BASOCELLULARE INFILTRANTE IL DERMA R...,44
49859,2010-12-30 11:37:10,1 - ADENOMA TUBULO - VILLOSO INTESTINALE CON D...,18
49860,2010-12-30 11:37:10,CARCINOMA SQUAMOCELLULARE BEN DIFFERENZIATO IN...,44


In [65]:
dfValid

,data,text,y
49874,2010-12-30 11:37:10,CARCINOMA BASOSQUAMOCELLULARE ULCERATO INFILTR...,44
48566,2010-12-30 11:37:10,CARCINOMA A CELLULE BASALI . LESIONE NON COMPR...,44
45980,2010-12-30 11:37:10,ISOLATI FRAMMENTI RIFERIBILI AD ADENOMA TUBULA...,20
47380,2010-12-30 11:37:10,CARCINOMA BASOCELLULARE DI TIPO SUPERFICIALE ....,44
48379,2010-12-30 11:37:10,RETTORRAGIA . ADENOMA TUBULO - VILLOSO INTESTI...,20
...,...,...,...
75683,2012-03-06 13:09:47,RESIDUE ESPRESSIONI DI CARCINOMA BASOCELLULARE...,44
79907,2012-03-06 13:09:59,CARCINOMA A CELLULE CHIARE RENALE ( FUHRMAN : ...,64
74879,2012-03-06 13:10:20,NEVO MELANOCITICO COMPOSTO DI TIPO CONGENITO A...,44
74380,2012-03-06 13:10:48,ADENOCARCINOMA INTESTINALE A MEDIO GRADO DI DI...,18


In [66]:
dfTest

,data,text,y
78863,2012-03-06 13:11:25,CARCINOMA BASOCELLULARE ; MARGINI NEGATIVI ( A...,44
75006,2012-03-06 13:11:45,FRAMMENTI ISTOLOGICI RIFERIBILI AD ADENOCARCIN...,18
75935,2012-03-06 13:11:55,CARCINOMA INFILTRANTE DI TIPO COLLOIDE EXTRACE...,50
79952,2012-03-06 13:12:18,PRESENZA DI UN SINGOLO FOCOLAIO DI ADENOCARCIN...,61
79484,2012-03-06 13:12:31,ADENOCARCINOMA ENDOMETRIALE DI BASSO GRADO ( G...,54
...,...,...,...
84732,2014-02-13 08:51:32,CARCINOMA DUTTALE INFILTRANTE ( G2 ) DELLA MAM...,50
47068,2014-02-13 11:20:50,"CARCINOMA DUTTALE IN SITU NAS , PRESENZA DI CA...",50
78074,2014-02-14 12:00:33,1 - AGOBIOPSIA PROSTATICA DELLA LUNGHEZZA COMP...,61
1000,2014-03-10 12:26:26,CA . GASTRICO . ADENOCARCINOMA GASTRICO MODERA...,16


In [67]:
collections.Counter(dfTrain.data.apply(lambda d: d.year))

Counter({1992: 2,
         1996: 1,
         1997: 2,
         1998: 1,
         1999: 4,
         2000: 13,
         2001: 95,
         2002: 30,
         2003: 325,
         2004: 1506,
         2005: 9082,
         2006: 8371,
         2007: 17004,
         2008: 7739,
         2009: 1108,
         2010: 5822})

In [68]:
collections.Counter(dfValid.data.apply(lambda d: d.year))

Counter({2010: 850, 2011: 2685, 2012: 13500})

In [69]:
collections.Counter(dfTest.data.apply(lambda d: d.year))

Counter({2012: 16688, 2013: 341, 2014: 6})

In [70]:
cTrain = collections.Counter(dfTrain.y)
cTrain.most_common()

[(44, 11506),
 (50, 6011),
 (18, 5206),
 (61, 3969),
 (67, 3342),
 (34, 3182),
 (16, 2401),
 (42, 2097),
 (20, 1705),
 (64, 1265),
 (54, 1078),
 (53, 884),
 (56, 703),
 (73, 631),
 (32, 626),
 (25, 574),
 (19, 508),
 (71, 503),
 (22, 407),
 (49, 325),
 (80, 282),
 (70, 256),
 (77, 253),
 (24, 213),
 (15, 212),
 (62, 204),
 (6, 154),
 (51, 150),
 (23, 149),
 (2, 146),
 (17, 145),
 (75, 139),
 (38, 127),
 (21, 118),
 (66, 106),
 (72, 97),
 (68, 97),
 (0, 88),
 (48, 83),
 (26, 81),
 (7, 81),
 (9, 78),
 (65, 78),
 (11, 59),
 (30, 57),
 (60, 52),
 (69, 52),
 (40, 48),
 (10, 48),
 (57, 48),
 (31, 40),
 (55, 40),
 (8, 40),
 (74, 37),
 (1, 35),
 (41, 34),
 (5, 34),
 (12, 32),
 (4, 27),
 (37, 25),
 (52, 25),
 (3, 23),
 (13, 22),
 (76, 17),
 (14, 15),
 (63, 15),
 (47, 13),
 (33, 3),
 (58, 3),
 (39, 1)]

In [71]:
cValid = collections.Counter(dfValid.y)
cValid.most_common()

[(44, 5239),
 (50, 1905),
 (18, 1889),
 (61, 1866),
 (34, 832),
 (16, 718),
 (20, 554),
 (54, 459),
 (42, 395),
 (67, 371),
 (64, 363),
 (53, 203),
 (77, 186),
 (56, 185),
 (19, 159),
 (73, 149),
 (32, 146),
 (26, 115),
 (71, 108),
 (25, 106),
 (49, 103),
 (22, 99),
 (15, 65),
 (2, 61),
 (80, 54),
 (21, 53),
 (17, 49),
 (24, 44),
 (51, 42),
 (62, 41),
 (6, 38),
 (38, 35),
 (23, 33),
 (7, 28),
 (9, 27),
 (0, 24),
 (41, 23),
 (30, 20),
 (66, 19),
 (48, 17),
 (60, 17),
 (65, 16),
 (11, 15),
 (40, 13),
 (57, 12),
 (52, 11),
 (75, 11),
 (69, 11),
 (8, 11),
 (5, 10),
 (70, 9),
 (4, 9),
 (74, 8),
 (13, 8),
 (12, 7),
 (55, 6),
 (3, 6),
 (10, 5),
 (1, 5),
 (76, 4),
 (72, 3),
 (63, 3),
 (31, 3),
 (47, 3),
 (14, 2),
 (58, 1),
 (37, 1),
 (33, 1),
 (68, 1)]

In [72]:
cTest = collections.Counter(dfTest.y)
cTest.most_common()

[(44, 4249),
 (18, 2152),
 (67, 1801),
 (50, 1594),
 (61, 903),
 (16, 669),
 (53, 642),
 (42, 615),
 (20, 579),
 (34, 497),
 (54, 358),
 (64, 325),
 (56, 299),
 (77, 264),
 (25, 166),
 (32, 163),
 (73, 148),
 (80, 146),
 (49, 126),
 (22, 118),
 (19, 115),
 (26, 108),
 (71, 65),
 (24, 60),
 (21, 53),
 (62, 52),
 (17, 52),
 (23, 46),
 (51, 46),
 (15, 46),
 (65, 44),
 (48, 42),
 (38, 38),
 (2, 34),
 (66, 32),
 (70, 31),
 (30, 27),
 (9, 27),
 (6, 25),
 (7, 25),
 (11, 24),
 (75, 21),
 (41, 19),
 (0, 18),
 (5, 17),
 (60, 15),
 (40, 15),
 (31, 14),
 (69, 11),
 (52, 8),
 (8, 8),
 (1, 7),
 (55, 7),
 (4, 7),
 (14, 7),
 (57, 6),
 (47, 6),
 (10, 6),
 (68, 6),
 (13, 6),
 (3, 5),
 (12, 4),
 (72, 4),
 (37, 4),
 (76, 3),
 (33, 2),
 (74, 2),
 (63, 1)]

In [73]:
cTest[44]

4249

In [74]:
toRemove = set()
for i in cTrain.keys():
    if not i in cTest.keys():
        toRemove.add(i)
    

In [75]:
toRemove

{39, 58}

In [76]:
for i in cValid.keys():
    if not i in cTest.keys() or not i in cTrain.keys():
        toRemove.add(i)

In [77]:
toRemove

{39, 58}

In [78]:
for i in cTest.keys():
    if not i in cTrain.keys():
        toRemove.add(i)

In [79]:
toRemove

{39, 58}

In [80]:
dfTrain = dfTrain[~dfTrain.y.isin(toRemove)]

In [81]:
dfValid = dfValid[~dfValid.y.isin(toRemove)]

In [82]:
dfTest = dfTest[~dfTest.y.isin(toRemove)]

In [83]:
corpus = {
    'train':{
        'text':dfTrain['text'].values,
        'sede1':dfTrain['y'].values,
    },
    'valid':{
        'text':dfValid['text'].values,
        'sede1':dfValid['y'].values,
    },
    'test':{
        'text':dfTest['text'].values,
        'sede1':dfTest['y'].values,
    },
}

In [84]:
count = collections.Counter(np.concatenate((corpus['train']['sede1'], corpus['valid']['sede1'], corpus['test']['sede1'])))

In [85]:
values = sorted(count.keys())

In [86]:
wordVectorsText = np.concatenate((dfUnlabelled.text.values, dfTrain.text.values, dfValid.text.values))

In [87]:
wordVectorsText[:10]

array(['PAZIENTE OPERATO DI MELANOMA 4 ANNI FA . PROLIFERAZIONE A CELLULE ROTONDE DA DEFINIRE DOPO INCLUSIONE . ESAME ISTOLOGICO AL CONGELATORE .',
       'ADENOMA TUBULARE INTESTINALE . POLIPO DEL SIGMA , DIAMETRO MAGGIORE 8 MM .',
       'FIBROLIPOMA ( 1 - 2 ) ( 1 - 2 ) FRAMMENTI DI ASPETTO LIPOMATOSO DI 3CM COMPLESSIVAMENTE .',
       'QUADRO ISTOMORFOLOGICO ED IMMUNOISTOCHIMICO COMPATIBILI CON LINFOMA GASTRICO ( VEDI ES . K 331 / 99 )',
       'QUADRO ISTOLOGICO E IMMUNOISTOCHIMICO ( VEDI ES . K 209 / 99 ) COMPATIBILE CON LLC MT',
       'ESAME INTRAOPERATORIO E PER INCLUSIONE : TUMORE ADENOMATOIDE . MT - FB - FS NODO DEL DIAMETRO MASSIMO DI CM 0 , 7 .',
       "NEOPLASIA DEL POLO INFERIORE DEL RENE SINISTRO . CARCINOMA A CELLULE RENALI CHIARE . STRUTTURE DELL ' ILO INDENNI . PT2 . 2 BB RENE SINISTRO DI CM 12 * 6 * 5 CON NEOPLASIA DI CM 5 .",
       'LIPOMA . MT',
       'ELEMENTI EPITELIALI ATIPICI COMPATIBILI CON CARCINOMA . QUATTRO VETRINI .',
       'ESAME INTRAOPERATORIO E PER

In [88]:
np.random.shuffle(wordVectorsText)

In [89]:
wordVectorsText[:10]

array(['2 ) PROLIPI FIBROGHIANDOLARI ENDOMETRIALI . 2 ) POLIPI ENDOMETRIALI .',
       'CARCINOSI PERITONEALE QUADRO CITOLOGICO POSITIVO PER ADENOCARCINOMA . 10 ML DI LIQUIDO ASCITICO CON CUI SI ALLESTICONO 4 PREPARATI COLORATI SECONDO PAPANICOLAU .',
       'NEGATIVO ( 0 ) ( KIT DAKO ) . HERCEP TEST SU INCLUSIONE N ¦ 1965 / 08',
       'MALATTIA DI HODGKIN A TIPO CELLULARITÀ MISTA . DUE LINFONODI LATEROCERVICALI DESTRI DI MM . 8X5X5 E MM . 6X5X4 .',
       'MODICO NUMERTO DI CELLULE CILINDRICHE . SPAZZOLAMENTO BRONCHIALE .',
       'PAZIENTE GASTRORESECATO NEL 1998 . 01 ; 03 : IPERPLASIA DELLA MUCOSA . 02 : ADENOMA TUBULARE ( FRAMMENTI ) CON DISPLASIA DI BASSO GRADO ( DISPLASIA LIEVE ) .',
       'LESIONE PIGMENTATA DELLA REG . SCAPOLARE DX NEVO MELANOCITICO DERMICO , CONGENITO .',
       'CUTE SOPRACLAVEARE SINISTRA A : NEVO INTRADERMICO . NEOFORMAZIONE PIGMENTATA DI 0 . 4 CM',
       'IL MATERIALE INVIATO , INTERAMENTE ESAMINATO , È COSTITUITO DA TESSUTO EMOPOIETICO MIDOLLARE , MATE

# write

In [90]:
pickle.dump(corpus, open(fileSaveCorpus, 'wb'))
pickle.dump(values, open(fileSaveValues, 'wb'))
np.savetxt(fileSaveTrainWordVectors, wordVectorsText, delimiter=" ", fmt="%s")